In [1]:
import dash_bootstrap_components as dbc
from dash import Input, Output, dcc, html
from explainerdashboard import ClassifierExplainer, ExplainerDashboard

/Users/antwnhsvellopoulos/miniconda3/lib/python3.7/site-packages/dash_bootstrap_components/table.py:1: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html

The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`


In [2]:
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score
from sklearn.feature_selection import SelectKBest,mutual_info_classif
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN

In [3]:
from dash_bootstrap_components.themes import FLATLY

## Importing the data 

In [4]:
data_model = pd.read_csv("../data/model_data")

In [5]:
data_model = data_model.drop(columns = 'Unnamed: 0', axis = 1)

In [6]:
data_model = data_model.rename(columns={'PreferredPaymentMode_1.0': 'PreferredPaymentMode_1',
                                'PreferredPaymentMode_3.0': 'PreferredPaymentMode_3',
                                 'PreferredPaymentMode_4.0':'PreferredPaymentMode_4'})

## Oversampling

In [7]:
X = data_model.drop('Churn', axis = 1)
y = data_model['Churn']

ada = ADASYN(sampling_strategy='minority', n_neighbors=5, n_jobs=-1)

X_res, y_res = ada.fit_resample(X,y)

y_res = pd.DataFrame(y_res, columns = ['Churn'])

## Scaling

In [8]:
transformer = RobustScaler()

X_tr = transformer.fit_transform(X_res)

X_tr_df = pd.DataFrame(X_tr, columns = X_res.columns)

X_train, X_test, y_train, y_test = train_test_split(X_tr_df, y_res, test_size=0.33, random_state=42)

## creating the dashboard

The next step is to create an interactive dashboard to communicate our findings with the rest of the team. We will create our dashboard using a library called explainerdashboard. explainerdashboard is a library for quickly building interactive dashboards for analyzing and explaining the predictions and workings of (scikit-learn compatible) machine learning models.

In [9]:
model = GradientBoostingClassifier(max_depth=3, n_estimators = 1600, learning_rate= 1)

In [10]:
model.fit(X_train,  y_train.values.ravel())

GradientBoostingClassifier(learning_rate=1, n_estimators=1600)

In [11]:
explainer = ClassifierExplainer(model, X_test, y_test, model_output='logodds')

Generating self.shap_explainer = shap.TreeExplainer(model)


In [16]:
db = ExplainerDashboard(explainer, title="model explainer", whatif=False)

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...


In [17]:
db.to_yaml("dashboard.yaml", explainerfile="explainer.joblib", dump_explainer=True)

Dumping configuration .yaml to /Users/antwnhsvellopoulos/Desktop/GitHub/E_commerce_customer_churn_prediciton/Dashboard/dashboard.yaml...
Dumping explainer to /Users/antwnhsvellopoulos/Desktop/GitHub/E_commerce_customer_churn_prediciton/Dashboard/explainer.joblib...
